API safe request rate is limit 1 request per second.

In [1]:
!pip install requests
!pip install diskcache

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
import base64
import json
import os
from datetime import datetime
import pytz
import diskcache
import urllib.parse

In [3]:
start_date = datetime.fromisoformat('2019-01-01T11:11:11+00:00')
now_date = datetime.utcnow().replace(tzinfo=pytz.utc)
last_update_date = None

In [4]:
project_dir = os.path.abspath('')

API = {
    "time_entries" : "https://api.track.toggl.com/api/v9/me/time_entries",
    "get_current_time_entry": "https://api.track.toggl.com/api/v9/me/time_entries/current", 
    "me_info" : "https://api.track.toggl.com/api/v9/me",
    "projects_me" : "https://api.track.toggl.com/api/v9/me/projects",
    "projects_ws" : "https://api.track.toggl.com/api/v9/workspaces/4143224/projects",
    "workspaces" : "https://api.track.toggl.com/api/v9/workspaces",
}

with open(f"{project_dir}/api_key.txt") as f:
    API_token = f"{f.read()}:api_token"
API_token_base64 = base64.b64encode(API_token.encode('ascii')).decode('ascii')

In [5]:
project_id_names = diskcache.Cache(f"{project_dir}/project_id_names.cache")
project_id_names.clear()

0

In [6]:
def fetch(url, headers={}):
    headers={"Content-Type": "application/json", 
                                    "Host": "api.track.toggl.com",
                                    "authorization": f"Basic {API_token_base64}", 
                                    **headers}
    r = requests.get(url, headers=headers)
    return json.loads(r.content.decode('utf-8'))

In [7]:
def fetch_project(project_id):
    return fetch(f"{API['projects_ws']}/{project_id}")

In [8]:
def filter_dict(d, keys):
    return list(map(lambda x: {k: v for k, v in x.items() if k in keys}, d))

In [36]:
def get_time_entries(start_date=now_date, end_date=now_date):
    url_exts = '?'
    url_exts += f"start_date={urllib.parse.quote('2022-11-10T00:53:28-02:00')}"
    url_exts += f"&end_date={urllib.parse.quote('2022-11-10T00:53:28-02:00')}"
    print(url_exts)
    time_entries = fetch(API['time_entries'] + url_exts)
    
    print('time_entries:', time_entries)
    
    for time_entry in time_entries:
        print(time_entry)
        print(type(time_entry))
        print(time_entry['project_id'])
        project_id = time_entry['project_id']
        if project_id not in project_id_names:
            project_id_names[project_id] = fetch_project(project_id)
            print('yoyo')
        time_entry['project_name'] = project_id_names[project_id]['name']
        
    return time_entries

In [37]:
def update_current_time_entry_duration(time_entry):
    time_entry["duration"] = (datetime.utcnow().replace(tzinfo=pytz.utc) - \
                              datetime.fromisoformat(time_entry["start"])).seconds
    return time_entry

In [38]:
start_date = datetime.fromisoformat('2022-11-10T23:53:28.000000+00:00')
end_date = datetime.fromisoformat('2022-11-13T23:53:28+00:00')
print(start_date.isoformat())
get_time_entries(start_date, end_date)[-1]

2022-11-10T23:53:28+00:00
?start_date=2022-11-10T00%3A53%3A28-02%3A00&end_date=2022-11-10T00%3A53%3A28-02%3A00
time_entries: error parsing date 2022-11-10T00:53:28-02:00
e
<class 'str'>


TypeError: string indices must be integers